In [1]:
import sys
sys.path.insert(0, "/home/dzigen/Desktop/nlp_service/")
from src.DocumentsSummarizer.utils import SummarizerConfig
import requests
import json


In [34]:
def llm_question(config: SummarizerConfig) -> str:

    url = 'http://localhost:11434/api/chat'
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": "Ты ассистент, который позволяет группировать слова по их смыслу и давать каждой группе её общее название. Все ответы ты генерируешь на русском языке и без грамматических ошибок. Давать ответ ты должен в следующем формате: \
                1. <общая тема 1>: <слово 1>, ..., <слово k>; \
                2. <общая тема 2>: <слово k+1>, ..., <слово k+m>; \
                ...\
                N. <общая тема N>: <слово k+m+1>, ..., <слово k+m+t>.\
                \
                Пример: \
                    Запрос: \
                    1. яблоко;\
                    2. помидор;\
                    3. апельсин;\
                    4. огурец.\
                    Ответ:\
                    1. фрукт: яблоко, апельсин;\
                    2. овощ: помидор, огурец."
            },
            {
                "role": "user",
                "content": '''
                1. колбаса;
                2. туалетная бумага;
                3. хлеб;
                4. зубная щётка;
                5. экскаватор;
                6. Пушкин;
                7. Черчилль;
                8. Есенин;
                9. Гитлер;
                10. Гоголь;
                11. Сталин.
                '''
            }
        ],
        "stream": config.stream,
        "options": {
            "num_keep": config.num_keep,
            "seed": config.seed,
            "num_predict": config.num_predict,
            "top_k": config.top_k,
            "top_p": config.top_p,
            "tfs_z": config.tfs_z,
            "typical_p": config.typical_p,
            "repeat_last_n": config.repeat_last_n,
            "temperature": config.temperature,
            "repeat_penalty": config.repeat_penalty,
            "presence_penalty": config.presence_penalty,
            "frequency_penalty": config.frequency_penalty,
            "mirostat": config.mirostat,
            "mirostat_tau": config.mirostat_tau,
            "mirostat_eta": config.mirostat_eta,
            "penalize_newline": config.penalize_newline,
            #"stop": config.stop,
            "numa": config.numa,
            "num_ctx": config.num_ctx,
            "num_batch": config.num_batch,
            "num_gpu": config.num_gpu,
            "main_gpu": config.main_gpu,
            "low_vram": config.low_vram,
            "f16_kv": config.f16_kv,
            "vocab_only": config.vocab_only,
            "use_mmap": config.use_mmap,
            "use_mlock": config.use_mlock,
            "num_thread": config.num_thread
        }
    }

    response = requests.post(url, json=data, stream=True)
    result = ''
    if response.status_code == 200:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                string_response = chunk.decode('utf-8')
                json_data = json.loads(string_response)

                # Извлечение значения content из JSON-данных
                if 'message' in json_data and 'content' in json_data['message']:
                    content = json_data['message']['content']
                    result += json_data['message']['content']
                    print(content, end='')
                else:
                    print("Content field not found in JSON response.")
        print('\n')

    else:
        print("Failed to get a successful response. Status code:", response.status_code)

    return result

In [35]:
config = SummarizerConfig()
result = llm_question(config)
#print(result)

1. историческая личность: Пушкин, Черчилль, Есенин, Гитлер;
2. хлебобулочное изделие и бытовая вещь: колбаса, хлеб, зубная щётка;
3. строительное оборудование: экскаватор;

